In [ ]:
import pandas as pd
import os
import csv
os.chdir("C:\\Study Material\\Thesis\\Prompt_Tuning_v2\\Tables_with_3_rows")
df = pd.read_csv("1.tsv", delimiter='\t')
df

In [ ]:
list_of_rows = []
read_dir = "C:\\Study Material\\Thesis\\Prompt_Tuning_v2\\Tables_with_3_rows"
for filename in range(1,343):
    read_path = read_dir + "\\" + str(filename) + ".tsv"
    df = pd.read_csv(read_path, delimiter='\t')
    for row_id in range(0,len(df)):
        individual_row = [df.iloc[row_id,0],df.iloc[row_id,1],df.iloc[row_id,2],df.iloc[row_id,3],df.iloc[row_id,4]]
        list_of_rows.append(individual_row)

In [ ]:
len(list_of_rows)

In [ ]:
write_dir = "C:\\Study Material\\Thesis\\Prompt_Tuning_v2\\Final_tables_with_2_rows"
current_filename = 0
for i in range(0,len(list_of_rows),2):
    current_index = i
    current_filename+=1
    path_write = write_dir + "\\" + str(current_filename) + ".tsv"
    with open(path_write,"w", newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter='\t')
        individual_row = ['Disease','Background','Key Symptoms','Key Signs','Additional information']
        csv_writer.writerow(individual_row)
        
        individual_row = list_of_rows[current_index]
        csv_writer.writerow(individual_row)
        
        individual_row = list_of_rows[current_index+1]
        csv_writer.writerow(individual_row)      
        
        
# Ignore error, tables are being created

In [ ]:
# Function to linearize table
import os

def convert_table_to_linearized(path_read):
#     table_to_linearize = pd.read_csv(path_read,encoding='utf-8',warn_bad_lines=True, error_bad_lines=False)
    table_to_linearize = pd.read_csv(path_read,encoding='utf-8', delimiter='\t')
    list_of_columns = table_to_linearize.columns
#     table_to_linearize = pd.read_csv(path_read,encoding='utf-8')
    number_of_rows = table_to_linearize.shape[0]
    number_of_columns = table_to_linearize.shape[1]
    linearized_table = ''
    for i in range(0,number_of_rows):
        linearized_table+= 'Row '+str(i+1)+' is : '
        for j in range(0,number_of_columns):
            linearized_table+= str(table_to_linearize.columns[j]).strip()+' is : '+str(table_to_linearize.iloc[i,j]).strip()
            if number_of_rows>0 and j<(number_of_columns - 1) :
               linearized_table+= ' ; ' 
        linearized_table+= '. '
    return linearized_table

os.chdir("C:\\Study Material\\Thesis\\Prompt_Tuning_v2\\Final_tables_with_2_rows")

read_dir = os.getcwd()
path_read = read_dir +"\\"+ "1.tsv"  
a = convert_table_to_linearized(path_read)
print(a)

In [ ]:
list_of_table_folder_paths =[]
table_directory = ("C:\\Study Material\\Thesis\\Prompt_Tuning_v2\\Final_tables_with_2_rows\\")
for i in range(1,514):
    table_path = table_directory + str(i) + ".tsv"
    list_of_table_folder_paths.append(table_path)

In [ ]:
list_of_linearized_tables = []
for i in range(0,513):    
    try: 
        linearized_table = convert_table_to_linearized(list_of_table_folder_paths[i])
        print(i," Successfully linearized the table: ",list_of_table_folder_paths[i]) 
        list_of_linearized_tables.append(linearized_table)
    except Exception as e:
        print("Skipping table:", list_of_table_folder_paths[i])
        print("Exception:", e)

In [ ]:
os.chdir("C:\\Study Material\\Thesis\\Prompt_Tuning_v2")
import json
linearized_tables_in_pre_json_format = []
for i in range(0,len(list_of_linearized_tables)):
     linearized_tables_in_pre_json_format.append({
    'id': (i+1),
    'contents': list_of_linearized_tables[i]
    })

with open('symbol_tables_linearized_contents.json', 'w') as outfile:
    json.dump(linearized_tables_in_pre_json_format,outfile)

In [ ]:
# Import CLEANED linearized contents of table
import json
linearized_table_contents = []

os.chdir("C:\\Study Material\\Thesis\\Prompt_Tuning_v2")
f = open('cleaned_symbol_tables_linearized_contents.json', 'r', errors="ignore")
json_object = json.load(f)
for table_data in json_object:    
    print(table_data['id'])
    print(table_data['contents'])
    linearized_table_contents.append(table_data['contents'])
f.close()

In [ ]:
import os
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

average = 0
total =0
count=0
max_len = 0
for table_id in range(0,len(linearized_table_contents)):
    tokens = tokenizer(linearized_table_contents[table_id])
    print(len(tokens['input_ids']))
    average+=len(tokens['input_ids'])
    total+=1
    if (len(tokens['input_ids'])<512):
        count+=1
    else:
        print('Table: ',(table_id+1),' token count: ',len(tokens['input_ids']))
    if (max_len<len(tokens['input_ids'])):
        max_len = len(tokens['input_ids'])
        
average= average/total
print('Average: ',average)
print('Count of tables less than 512: ',count)
print('max_len: ',max_len)

In [ ]:
len(linearized_table_contents)